# Mejorando $R^2$ y el $R^2$ ajustado

En este notebook es una extencion del anterior ya que en este me quiero centrar en mejorar el $R^2$ y el $R^2$ ajustado con la finalidad de obtener un mejor modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import statsmodels.api as sm
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Cargamos datos y preprocesamiento inicial (del código anterior) 
df_laptops = pd.read_csv("data/df_model_ready.csv")

In [2]:
# Las variables con las que trabajaremos
numerical_boxcox_features = [
    'Inches_BoxCox', 'Ram_BoxCox', 'Weight_BoxCox', 'CPU_freq_BoxCox',
    'PrimaryStorage_BoxCox', 'SecondaryStorage_BoxCox', 'ScreenPixels_BoxCox'
]
binary_features = [
    'Touchscreen', 'IPSpanel', 'RetinaDisplay'
]
categorical_features = [
    'Company', 'TypeName', 'OS', 'PrimaryStorageType',
    'SecondaryStorageType', 'CPU_company', 'GPU_company'
]
target_variable = 'Price_euros_BoxCox'
all_features = numerical_boxcox_features + binary_features + categorical_features

df_encoded = pd.get_dummies(df_laptops, columns=categorical_features, drop_first=True)

X = df_encoded.drop(columns=[target_variable])
y = df_encoded[target_variable]

X = X.loc[:, (X != 0).any(axis=0)]
X = X.dropna(axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamos el modelo ORIGINAL para comparación
print("Entrenando el modelo ORIGINAL para obtener sus métricas...")
model_original = LinearRegression()
model_original.fit(X_train, y_train)

# Calculamos métricas del modelo ORIGINAL
y_pred_train_original = model_original.predict(X_train)
r2_train_original = r2_score(y_train, y_pred_train_original)
mae_train_original = mean_absolute_error(y_train, y_pred_train_original)
mse_train_original = mean_squared_error(y_train, y_pred_train_original)
rmse_train_original = np.sqrt(mse_train_original)

n_train_original = len(y_train)
k_train_original = X_train.shape[1]
r2_adjusted_train_original = 1 - ((1 - r2_train_original) * (n_train_original - 1)) / (n_train_original - k_train_original - 1)

y_pred_test_original = model_original.predict(X_test)
r2_test_original = r2_score(y_test, y_pred_test_original)
mae_test_original = mean_absolute_error(y_test, y_pred_test_original)
mse_test_original = mean_squared_error(y_test, y_pred_test_original)
rmse_test_original = np.sqrt(mse_test_original)

n_test_original = len(y_test)
k_test_original = X_test.shape[1]
r2_adjusted_test_original = 1 - ((1 - r2_test_original) * (n_test_original - 1)) / (n_test_original - k_test_original - 1)


Entrenando el modelo ORIGINAL para obtener sus métricas...


Hasta aqui no hay nada que no hubieramos visto en el anterior notebook ; pero ahora aplicaremos un filtro de baja varianza y tambien eliminaremos las caracteristicas que presenten una alta multicolinealidad

In [3]:
# Aplicar Selección de Características (Varianza y VIF) ---
# Paso de Varianza
print("\nAplicando filtro por baja varianza...")
selector = VarianceThreshold(threshold=0.001)
num_cols_before_var = X_train.shape[1]
selector.fit(X_train)
columns_to_keep_var = X_train.columns[selector.get_support()]
X_train_filtered_var = X_train[columns_to_keep_var]
X_test_filtered_var = X_test[columns_to_keep_var]
num_cols_after_var = X_train_filtered_var.shape[1]
print(f"Características eliminadas por baja varianza: {list(set(X_train.columns) - set(X_train_filtered_var.columns))}")
print(f"Total de columnas eliminadas por baja varianza: {num_cols_before_var - num_cols_after_var}")

# Paso de VIF
def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = df.columns
    if df.shape[1] == 0:
        return pd.DataFrame(columns=["feature", "VIF"])

    try:
        data_for_vif = df.astype(float)
    except ValueError as e:
        print(f"Error al convertir DataFrame a float para VIF: {e}")
        print("Tipos de datos de las columnas:", df.dtypes)
        raise

    data_for_vif = data_for_vif.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

    if data_for_vif.shape[1] == 0:
        return pd.DataFrame(columns=["feature", "VIF"])

    vif_data["VIF"] = [variance_inflation_factor(data_for_vif.values, i) for i in range(data_for_vif.shape[1])]
    return vif_data.sort_values(by="VIF", ascending=False)

X_train_vif_processed = X_train_filtered_var.copy()
for col in X_train_vif_processed.columns:
    X_train_vif_processed[col] = pd.to_numeric(X_train_vif_processed[col], errors='coerce')
X_train_vif_processed = X_train_vif_processed.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

vif_threshold = 10
print("\nCalculando VIFs y eliminando características con alta multicolinealidad...")
initial_vif_cols = X_train_vif_processed.shape[1]

while True:
    if X_train_vif_processed.shape[1] == 0:
        print("No hay más características para calcular VIF. Saliendo del bucle.")
        break

    vifs = calculate_vif(X_train_vif_processed)

    if vifs.empty:
        print("La tabla de VIFs está vacía. Saliendo del bucle.")
        break

    if vifs.iloc[0]["VIF"] > vif_threshold:
        col_to_drop = vifs.iloc[0]["feature"]
        if col_to_drop in X_train_vif_processed.columns:
            X_train_vif_processed = X_train_vif_processed.drop(columns=[col_to_drop])
            print(f"Eliminando '{col_to_drop}' debido a VIF alto ({vifs.iloc[0]['VIF']:.2f})")
        else:
            print(f"Advertencia: La columna '{col_to_drop}' ya no existe en X_train_vif_processed. Saliendo del bucle VIF.")
            break
    else:
        break

final_features_after_vif = X_train_vif_processed.columns
X_train_final = X_train_filtered_var[final_features_after_vif]
X_test_final = X_test_filtered_var[final_features_after_vif]

print(f"\nCaracterísticas finales después de la selección por VIF: {list(final_features_after_vif)}")
print(f"Número de características finales (después de varianza y VIF): {X_train_final.shape[1]}")
print(f"Total de columnas eliminadas por VIF: {initial_vif_cols - X_train_final.shape[1]}")



Aplicando filtro por baja varianza...
Características eliminadas por baja varianza: ['ScreenPixels_BoxCox', 'CPU_company_Samsung', 'GPU_company_ARM', 'Company_Huawei']
Total de columnas eliminadas por baja varianza: 4

Calculando VIFs y eliminando características con alta multicolinealidad...
Eliminando 'OS_Mac OS X' debido a VIF alto (inf)


c:\Users\delhy.py\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


Eliminando 'SecondaryStorageType_No' debido a VIF alto (534.41)
Eliminando 'OS_Windows 10' debido a VIF alto (113.81)
Eliminando 'Inches_BoxCox' debido a VIF alto (100.52)
Eliminando 'PrimaryStorage_BoxCox' debido a VIF alto (96.30)
Eliminando 'Ram_BoxCox' debido a VIF alto (44.25)
Eliminando 'CPU_company_Intel' debido a VIF alto (31.23)
Eliminando 'PrimaryStorageType_SSD' debido a VIF alto (21.04)
Eliminando 'Weight_BoxCox' debido a VIF alto (16.24)

Características finales después de la selección por VIF: ['CPU_freq_BoxCox', 'SecondaryStorage_BoxCox', 'Touchscreen', 'IPSpanel', 'RetinaDisplay', 'Company_Apple', 'Company_Asus', 'Company_Chuwi', 'Company_Dell', 'Company_Fujitsu', 'Company_Google', 'Company_HP', 'Company_LG', 'Company_Lenovo', 'Company_MSI', 'Company_Mediacom', 'Company_Microsoft', 'Company_Razer', 'Company_Samsung', 'Company_Toshiba', 'Company_Vero', 'Company_Xiaomi', 'TypeName_Gaming', 'TypeName_Netbook', 'TypeName_Notebook', 'TypeName_Ultrabook', 'TypeName_Workstatio

In [4]:
# Reentrenamos el modelo con las características seleccionadas 
print("\nReentrenando el modelo con las características seleccionadas (MODELO OPTIMIZADO)...")
model_optimized = LinearRegression()
model_optimized.fit(X_train_final, y_train)
print("Modelo optimizado entrenado exitosamente.")

# Evaluamos el modelo optimizado 
y_pred_train_optimized = model_optimized.predict(X_train_final)
r2_train_optimized = r2_score(y_train, y_pred_train_optimized)
mae_train_optimized = mean_absolute_error(y_train, y_pred_train_optimized)
mse_train_optimized = mean_squared_error(y_train, y_pred_train_optimized)
rmse_train_optimized = np.sqrt(mse_train_optimized)

n_train_optimized = len(y_train)
k_train_optimized = X_train_final.shape[1]
r2_adjusted_train_optimized = 1 - ((1 - r2_train_optimized) * (n_train_optimized - 1)) / (n_train_optimized - k_train_optimized - 1)

y_pred_test_optimized = model_optimized.predict(X_test_final)
r2_test_optimized = r2_score(y_test, y_pred_test_optimized)
mae_test_optimized = mean_absolute_error(y_test, y_pred_test_optimized)
mse_test_optimized = mean_squared_error(y_test, y_pred_test_optimized)
rmse_test_optimized = np.sqrt(mse_test_optimized)

n_test_optimized = len(y_test)
k_test_optimized = X_test_final.shape[1]
r2_adjusted_test_optimized = 1 - ((1 - r2_test_optimized) * (n_test_optimized - 1)) / (n_test_optimized - k_test_optimized - 1)


Reentrenando el modelo con las características seleccionadas (MODELO OPTIMIZADO)...
Modelo optimizado entrenado exitosamente.


In [5]:
# Imprimimos la tabla comparativa de métricas 
print("\n# Comparación de Métricas del Modelo de Regresión Lineal\n")
print("| Métrica                                  | Original (Entrenamiento) | Optimizado (Entrenamiento) | Original (Prueba)  | Optimizado (Prueba)  |")
print("| ---------------------------------------: | -----------------------: | -------------------------: | -----------------: | --------------------:|")
print(f"| {'R-cuadrado (R²)':<40} | {r2_train_original:>24.4f} | {r2_train_optimized:>26.4f} | {r2_test_original:>18.4f} | {r2_test_optimized:>20.4f} |")
print(f"| {'R-cuadrado Ajustado (R²_adj)':<40} | {r2_adjusted_train_original:>24.4f} | {r2_adjusted_train_optimized:>26.4f} | {r2_adjusted_test_original:>18.4f} | {r2_adjusted_test_optimized:>20.4f} |")
print(f"| {'Error Absoluto Medio (MAE)':<40} | {mae_train_original:>24.4f} | {mae_train_optimized:>26.4f} | {mae_test_original:>18.4f} | {mae_test_optimized:>20.4f} |")
print(f"| {'Error Cuadrático Medio (MSE)':<40} | {mse_train_original:>24.4f} | {mse_train_optimized:>26.4f} | {mse_test_original:>18.4f} | {mse_test_optimized:>20.4f} |")
print(f"| {'Raíz del Error Cuadrático Medio (RMSE)':<40} | {rmse_train_original:>24.4f} | {rmse_train_optimized:>26.4f} | {rmse_test_original:>18.4f} | {rmse_test_optimized:>20.4f} |")


# Comparación de Métricas del Modelo de Regresión Lineal

| Métrica                                  | Original (Entrenamiento) | Optimizado (Entrenamiento) | Original (Prueba)  | Optimizado (Prueba)  |
| ---------------------------------------: | -----------------------: | -------------------------: | -----------------: | --------------------:|
| R-cuadrado (R²)                          |                   0.8365 |                     0.7001 |             0.8094 |               0.6814 |
| R-cuadrado Ajustado (R²_adj)             |                   0.8277 |                     0.6881 |             0.7603 |               0.6236 |
| Error Absoluto Medio (MAE)               |                   0.4394 |                     0.6090 |             0.4717 |               0.5954 |
| Error Cuadrático Medio (MSE)             |                   0.3195 |                     0.5862 |             0.3297 |               0.5512 |
| Raíz del Error Cuadrático Medio (RMSE)   |                   0.5652 |

### Conclusiones Finales del Modelo de Regresion Lineal Multiple 

Basándonos en las métricas motradas en la tabla, podemos concluir que la selección de características aplicada fue demasiado agresiva para este dataset y este modelo. Las características que fueron eliminadas (debido a baja varianza o alta multicolinealidad con el umbral de VIF elegido) contenían información predictiva valiosa que el modelo lineal utilizaba para realizar predicciones más precisas.

Por tanto nos quedaremos con el modelo original (el realizado en el anterior notebook) para la realizacion del proyecto que estamos realizando